## **Kubernetes Ports: The TSE Must-Know Cheat Sheet**

For your Google TSE interview, these are the ports you **must** have at mental recall. I've grouped them by function.

---

## **🔌 CONTROL PLANE PORTS (The Brains)**

| Component | Port | Protocol | Direction | Interview Context |
|-----------|------|----------|-----------|-------------------|
| **kube-apiserver** | **6443** | HTTPS | All nodes → Control Plane | **The most important port.** All kubectl commands, API calls, and component communication goes here. If this is down, cluster is dead. |
| **kube-apiserver (legacy)** | 443 | HTTPS | Same as above | Some distributions use 443, but 6443 is standard |
| **etcd** | **2379** | HTTPS | apiserver → etcd | etcd client API. Only apiserver should talk here. |
| **etcd peer** | 2380 | HTTPS | etcd ↔ etcd | etcd clustering/peer communication |
| **kube-scheduler** | 10259 | HTTPS | localhost | Scheduler health/leaderelection |
| **kube-controller-manager** | 10257 | HTTPS | localhost | Controller manager health/leaderelection |

**Interview Soundbite:** *"The API server on 6443 is the heart of Kubernetes. Everything goes through it—kubectl, controllers, even nodes. If I can't reach 6443, the cluster is effectively dead."*

---

## **📦 WORKER NODE PORTS (The Muscles)**

| Component | Port | Protocol | Direction | Purpose |
|-----------|------|----------|-----------|---------|
| **kubelet** | **10250** | HTTPS | Control Plane → Node | **Primary kubelet API.** Scheduler uses this to manage pods, exec into containers, get logs. |
| **kubelet (read-only)** | 10255 | HTTP | Internal | Deprecated read-only endpoint (metrics, health) |
| **kube-proxy** | 10256 | HTTP | Internal | Health check port |
| **NodePort Services** | **30000-32767** | TCP/UDP | External → Node | Default range for NodePort services |

**Critical TSE Knowledge:** 
- 10250 is where the control plane talks to nodes
- If 10250 is blocked, nodes are cordoned/NotReady
- `kubectl exec` and `kubectl logs` use this port

---

## **🌐 SERVICE MESH & ADD-ONS**

| Component | Port | Protocol | Purpose |
|-----------|------|----------|---------|
| **CoreDNS** | 53 | TCP/UDP | DNS resolution inside cluster |
| **CoreDNS metrics** | 9153 | HTTP | DNS metrics |
| **Ingress Controller (HTTP)** | 80 | HTTP | Default HTTP ingress |
| **Ingress Controller (HTTPS)** | 443 | HTTPS | Default HTTPS ingress |
| **Prometheus** | 9090 | HTTP | Metrics UI/API |
| **Grafana** | 3000 | HTTP | Dashboards |

---

## **🎯 THE ABSOLUTE MUST-MEMORIZE PORTS**

### **Top 5 Ports for TSE Interviews**

| # | Port | Component | Why You Must Know It |
|---|------|-----------|---------------------|
| **1** | **6443** | kube-apiserver | *"Cluster is down? First check: is 6443 responding?"* |
| **2** | **10250** | kubelet | *"Node issues? Can the control plane reach 10250?"* |
| **3** | **2379** | etcd | *"etcd health? Check 2379—but only from apiserver."* |
| **4** | **30000-32767** | NodePort range | *"External access problems? NodePort might be blocked."* |
| **5** | **53** | CoreDNS | *"DNS resolution failing? CoreDNS on 53."* |

---

## **🔍 DIAGNOSTIC FLOW WITH PORTS**

When troubleshooting connectivity:

```
kubectl get nodes -o wide  # Get node IPs

# Check API server
curl -k https://<control-plane>:6443/healthz

# Check kubelet on node
curl -k https://<node-ip>:10250/healthz

# Check etcd (from master)
curl --cacert /etc/kubernetes/pki/etcd/ca.crt \
     --cert /etc/kubernetes/pki/etcd/healthcheck-client.crt \
     --key /etc/kubernetes/pki/etcd/healthcheck-client.key \
     https://127.0.0.1:2379/health
```

---

## **🎤 INTERVIEW SOUNDBITES**

**When asked about API server unreachable:**
> *"First thing I check is connectivity to port 6443 on the control plane. If that's blocked, nothing works—kubectl, controllers, nodes. I'd verify firewall rules and that the apiserver process is listening on 6443."*

**When asked about node not ready:**
> *"I check if the kubelet is healthy on port 10250. The control plane uses this port to manage pods. If I can't reach it, the node can't report its status or receive work."*

**When asked about external access:**
> *"For NodePort services, I verify that the node's firewall allows the NodePort range (30000-32767). Many organizations block these ports by default."*

**When asked about DNS issues:**
> *"I check if CoreDNS is listening on port 53 and that network policies aren't blocking UDP/53 traffic between pods."*

---

## **📋 QUICK REFERENCE CARD**

```
CONTROL PLANE:
└─ 6443  - kube-apiserver (EVERYTHING goes here)
   ├─ 2379 - etcd (only apiserver talks here)
   ├─ 10259 - scheduler health
   └─ 10257 - controller-manager health

WORKER NODES:
├─ 10250 - kubelet API (control plane → node)
├─ 10255 - kubelet read-only (deprecated)
├─ 30000-32767 - NodePort services
└─ 10256 - kube-proxy health

CLUSTER SERVICES:
├─ 53    - CoreDNS (DNS resolution)
├─ 80    - HTTP ingress
└─ 443   - HTTPS ingress
```

**Remember:** 6443 is your best friend. 10250 is your second best. Everything else is context.